In [1]:
k = 4

In [2]:
import os
from tqdm import tqdm

import pandas as pd
import numpy as np

def generate_combinations(alphabet, k):
    if k == 0: return ['']
        
    combinations = []
    for char in alphabet:
        for suffix in generate_combinations(alphabet, k - 1):
            combinations.append(char + suffix)
    
    return combinations

In [3]:
set_kmer = generate_combinations(alphabet=['A', 'C', 'G', 'T'], k=k)
set_type = {kmer: np.float16 for kmer in set_kmer}

In [4]:
dfData = pd.read_csv(f'data/DATA_ITS_genus_{k}mer.csv', dtype=set_type)

In [5]:
dfData.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104500 entries, 0 to 104499
Columns: 258 entries, Genus to TTTT
dtypes: float16(256), object(2)
memory usage: 65.2 MB


In [6]:
dfData

,Genus,Species,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
0,Absidia,Absidia_spinosa,1.816406,0.237427,-0.946777,2.605469,-0.157227,0.632324,-0.157227,0.237427,...,-0.551758,-0.157227,1.421875,0.237427,0.237427,-0.551758,2.605469,1.026367,1.421875,10.890625
1,Absidia,Absidia_sp,3.476562,-0.510254,-0.067444,3.033203,-0.067444,-0.067444,-0.067444,0.375488,...,-0.510254,1.260742,1.260742,0.375488,1.260742,-0.067444,0.818359,2.589844,1.260742,9.679688
2,Absidia,Absidia_sp,1.476562,0.442383,-0.074707,1.993164,0.442383,-0.074707,0.959473,-0.074707,...,-0.591797,3.027344,-0.591797,-0.074707,0.959473,-0.074707,1.993164,1.993164,0.959473,3.027344
3,Absidia,Absidia_sp,2.394531,-0.622559,-0.622559,4.406250,-0.119812,-0.622559,-0.119812,-0.119812,...,-0.622559,2.896484,0.885742,-0.119812,2.394531,0.383057,0.885742,3.902344,2.394531,4.406250
4,Absidia,Absidia_sp,3.980469,-0.451660,-0.048798,2.771484,-0.048798,1.562500,-0.048798,0.756836,...,-0.451660,1.562500,-0.048798,-0.451660,1.160156,-0.048798,0.756836,3.173828,1.160156,10.023438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104495,Zyzygomyces,Zyzygomyces_bachmannii,3.101562,2.007812,-1.272461,2.007812,0.914062,-0.179443,0.367432,1.460938,...,-1.272461,-0.179443,1.460938,1.460938,1.460938,-0.726074,0.367432,-0.726074,1.460938,-0.179443
104496,Zyzygomyces,Zyzygomyces_bachmannii,3.097656,2.007812,-1.262695,2.007812,0.917969,-0.172485,0.372559,2.007812,...,-1.262695,-0.172485,1.462891,1.462891,1.462891,-0.717773,0.372559,-0.717773,1.462891,-0.172485
104497,Zyzygomyces,Zyzygomyces_bachmannii,3.507812,1.739258,-1.208984,2.328125,1.149414,-0.619629,0.559570,1.739258,...,-1.208984,-0.029938,1.739258,1.739258,1.149414,-0.619629,0.559570,-0.619629,1.739258,-0.029938
104498,Zyzygomyces,Zyzygomyces_bachmannii,3.105469,2.013672,-1.265625,2.013672,0.919922,-0.172852,0.373535,1.466797,...,-1.265625,-0.172852,1.466797,1.466797,1.466797,-0.719238,0.373535,-0.719238,1.466797,-0.172852


In [7]:
print('# genus  : ', len(dfData['Genus'].unique()))
print('# species: ', len(dfData['Species'].unique()))
print('# samples: ', dfData.shape[0])

# genus  :  1045
# species:  11954
# samples:  104500


In [8]:
import torch

print("PyTorch 버전:", torch.__version__)
print("CUDA 사용 가능 여부:", torch.cuda.is_available())

PyTorch 버전: 2.1.0
CUDA 사용 가능 여부: True


In [9]:
X = dfData.iloc[:, 2:].values
input_size = X.shape[1]

In [10]:
X.shape

(104500, 256)

In [11]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

y = dfData['Genus']
y_encoded = label_encoder.fit_transform(y)
num_classes = max(y_encoded) + 1
ansLabel = label_encoder.classes_

/disk1/01.Myeongkyu/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/disk1/01.Myeongkyu/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [12]:
import torch.nn as nn
import torch.optim as optim

In [13]:
class MyLeNet(nn.Module):
    def __init__(self, c1, c2, kernel, input_size, num_classes):
        super(MyLeNet, self).__init__()
        self.conv1 = nn.Conv1d(1, c1, kernel_size=kernel, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(c1, c2, kernel_size=kernel, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(((input_size - kernel + 1) // 2 - kernel + 1) // 2 * c2, 512)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [14]:
from torch.utils.data import DataLoader, TensorDataset

In [15]:
X_tensor = torch.from_numpy(X).float().unsqueeze(1)
y_tensor = torch.from_numpy(y_encoded).long()

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
num_samples = X.shape[0]

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

n_folds = 10
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

In [18]:
import time

In [19]:
import copy

from sklearn.metrics import accuracy_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

done_cnt = 0

# **** hyperparameters ****
for c1 in (16, 32):
    for c2 in (32, 64):
        for kernel in (3, 4):

# *************************
            n_iter = 0
            best_acc = 0
            
            train_acc_list = []
            val_acc_list = []
            test_acc_list = []
            time_list = []

            name = f'genus_LENET5_ITS_{k}mer_{c1},{c2},{kernel}'
            directory = 'new_results/' + name

            print(name)
            
            for train_index, test_index in kf.split(X_tensor, y_tensor):
                fold_test_answer_list = []
                n_iter += 1
                
                X_temp, X_test = X_tensor[train_index], X_tensor[test_index]
                y_temp, y_test = y_tensor[train_index], y_tensor[test_index]
            
                X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1)
                
                train_dataset = TensorDataset(X_train, y_train)
                val_dataset = TensorDataset(X_val, y_val)
                test_dataset = TensorDataset(X_test, y_test)
                
                batch_size = 1024
                train_DL = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                val_DL = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
                test_DL = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
            
                DL_dict = {'train': train_DL, 'val': val_DL}
                
                model = MyLeNet(c1=c1, c2=c2, kernel=kernel,
                                input_size=4**k, num_classes=num_classes)
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(model.parameters(), lr=0.001)
            
                model.to(device)
            
                num_epochs = 100
            
                epoch_train_acc_list = []
                epoch_val_acc_list = []
                epoch_test_acc_list = []
                epoch_test_answer_list = []
                epoch_time_list = []
                
                for epoch in tqdm(range(num_epochs)):
                    fold_test_answer_list.append(f'----- Epoch {epoch+1} -----\n')
                    time_st = time.time()
                    
                    for phase in ['train', 'val']:
                        if phase == 'train': model.train()
                        else: model.eval()
                        epoch_corrects = 0
                        
                        for inputs, labels in DL_dict[phase]:
                            inputs, labels = inputs.to(device), labels.to(device)
                            optimizer.zero_grad()
                
                            with torch.set_grad_enabled(phase == 'train'):
                                outputs = model(inputs)
                                _, preds = torch.max(outputs, 1)
                                loss = criterion(outputs, labels)
                
                                if phase == 'train':
                                    loss.backward()
                                    optimizer.step()
                
                                epoch_corrects += torch.sum(preds == labels.data)
                                
                        epoch_acc = epoch_corrects.double() / len(DL_dict[phase].dataset)
                        if phase == 'train': epoch_train_acc_list.append(str(epoch_acc.item()))
                        else: epoch_val_acc_list.append(str(epoch_acc.item()))
            
                    model.eval()
                    test_corrects = 0
                    with torch.no_grad():
                        for inputs, labels in test_DL:
                            inputs, labels = inputs.to(device), labels.to(device)
                            outputs = model(inputs)
                            _, preds = torch.max(outputs, 1)
                            test_corrects += torch.sum(preds == labels.data)
                        test_acc = test_corrects.double() / len(test_DL.dataset)
                        # print(f'Fold {n_iter} test Acc: {test_acc:.6f}, {time.time() - time_st} s') 
                    epoch_test_acc_list.append(str(test_acc.item()))
            
                    preds_list = preds.tolist()
                    label_list = labels.tolist()
                    for i in range(len(preds_list)):
                        epoch_test_answer_list.append(f'{label_list[i]},{preds_list[i]}')
            
                    epoch_time_list.append(str(time.time() - time_st))
                    fold_test_answer_list.append('\n'.join(epoch_test_answer_list))
                    fold_test_answer_list.append('\n')
            
                train_acc_list.append(','.join(epoch_train_acc_list) + '\n')
                val_acc_list.append(','.join(epoch_val_acc_list) + '\n')
                test_acc_list.append(','.join(epoch_test_acc_list) + '\n')
                
                time_list.append(','.join(epoch_time_list) + '\n')
            
                if not os.path.isdir(directory):
                    os.mkdir(directory)
            
                with open(directory + f'/Fold_{n_iter:02}.csv', 'w') as f:
                    f.write(''.join(fold_test_answer_list))
                
            
            with open(directory + '/train_acc.csv', 'w') as f:
                f.write(''.join(train_acc_list))
            
            with open(directory + '/val_acc.csv', 'w') as f:
                f.write(''.join(val_acc_list))
            
            with open(directory + '/test_acc.csv', 'w') as f:
                f.write(''.join(test_acc_list))
            
            with open(directory + '/times.csv', 'w') as f:
                f.write(','.join(time_list))

genus_LENET5_ITS_4mer_16,32,3


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:32<00:00,  3.04it/s]


genus_LENET5_ITS_4mer_16,32,4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:33<00:00,  3.02it/s]


genus_LENET5_ITS_4mer_16,64,3


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:33<00:00,  3.02it/s]


genus_LENET5_ITS_4mer_32,64,3


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:33<00:00,  3.03it/s]


genus_LENET5_ITS_4mer_32,64,4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:33<00:00,  3.00it/s]
